In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
import joblib

# Load dataset (with archetypes and masking logic)
df = pd.read_csv("MCA_Student_Performance_Balanced.csv")
subjects = [col.replace("_Minor1", "") for col in df.columns if "_Minor1" in col]

def prepare_features_targets(df, subjects, threshold, include_endsem=True):
    feature_columns = []
    target_columns = []
    for sub in subjects:
        feature_columns.extend([f"{sub}_Minor1", f"{sub}_Minor2"])
        if include_endsem:
            feature_columns.append(f"{sub}_EndSem")
    X = df[feature_columns]
    for sub in subjects:
        min1 = df[f"{sub}_Minor1"]
        min2 = df[f"{sub}_Minor2"]
        endsem = df[f"{sub}_EndSem"] if include_endsem else 0
        # For minors-only: threshold should be realistic for only two minors (e.g., 24 out of 40)
        # For minors+endsem: threshold higher (e.g., 60 or another realistic sum)
        total_score = min1 + min2 + (endsem if include_endsem else 0)
        is_missing = (min1 == 0) & (min2 == 0) & ((endsem == 0) if include_endsem else True)
        df[f"{sub}_IsWeak"] = np.where(is_missing, 0, (total_score < threshold).astype(int))
        target_columns.append(f"{sub}_IsWeak")
    y = df[target_columns]
    return X, y

def scale_and_split_save_scaler(X, y, scaler_filename):
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    joblib.dump(scaler, scaler_filename)
    return train_test_split(X_scaled, y, test_size=0.2, random_state=42)

def build_model(input_dim, output_dim):
    model = Sequential([
        Dense(64, input_dim=input_dim, activation="relu", kernel_regularizer=l2(0.001)),
        Dense(32, activation="relu", kernel_regularizer=l2(0.001)),
        Dropout(0.25),
        Dense(output_dim, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Model A (Minors only): threshold = 24 (for two minors, each max 20)
X_minor, y_minor = prepare_features_targets(df, subjects, 24, include_endsem=False)
X_train_A, X_test_A, y_train_A, y_test_A = scale_and_split_save_scaler(X_minor, y_minor, "minmax_scaler_minor.pkl")
model_A = build_model(X_train_A.shape[1], len(subjects))
history_A = model_A.fit(X_train_A, y_train_A, validation_data=(X_test_A, y_test_A), epochs=50, batch_size=16, verbose=1)
model_A.save("WeaknessPredictor_MinorsOnly.h5")

# Model B (Minors + EndSem): threshold = 60 (e.g. two minors + one endsem: each max 20/20/60)
X_full, y_full = prepare_features_targets(df, subjects, 60, include_endsem=True)
X_train_B, X_test_B, y_train_B, y_test_B = scale_and_split_save_scaler(X_full, y_full, "minmax_scaler_full.pkl")
model_B = build_model(X_train_B.shape[1], len(subjects))
history_B = model_B.fit(X_train_B, y_train_B, validation_data=(X_test_B, y_test_B), epochs=50, batch_size=16, verbose=1)
model_B.save("WeaknessPredictor_FullData.h5")

# --- Evaluation & Metrics ---
loss_A, acc_A = model_A.evaluate(X_test_A, y_test_A, verbose=0)
loss_B, acc_B = model_B.evaluate(X_test_B, y_test_B, verbose=0)
print(f"Model A (Minor-only): Accuracy = {acc_A*100:.2f}% | Loss = {loss_A:.4f}")
print(f"Model B (Full-data): Accuracy = {acc_B*100:.2f}% | Loss = {loss_B:.4f}")

print("\n📊 Model A (Minor-only) Results:")
print(f"Accuracy: {acc_A*100:.2f}% | Loss: {loss_A:.4f}")
y_pred_A = (model_A.predict(X_test_A) > 0.5).astype(int)
print("\n🔍 Overall Classification Report:")
print(classification_report(y_test_A.values.ravel(), y_pred_A.ravel(), target_names=["Not Weak", "Weak"]))
print(confusion_matrix(y_test_A.values.ravel(), y_pred_A.ravel()))

print("\n📊 Model B (Full-data) Results:")
print(f"Accuracy: {acc_B*100:.2f}% | Loss: {loss_B:.4f}")
y_pred_B = (model_B.predict(X_test_B) > 0.5).astype(int)
print("\n🔍 Overall Classification Report:")
print(classification_report(y_test_B.values.ravel(), y_pred_B.ravel(), target_names=["Not Weak", "Weak"]))
print(confusion_matrix(y_test_B.values.ravel(), y_pred_B.ravel()))

print("Minor-only label distribution:", np.bincount(y_minor.values.ravel()))
print("Full-data label distribution:", np.bincount(y_full.values.ravel()))

# --- Sample predictions for a test student ---
sample_A = np.expand_dims(X_test_A[0], axis=0)
pred_A = (model_A.predict(sample_A)[0] > 0.5).astype(int)
actual_A = y_test_A.iloc[0].values

sample_B = np.expand_dims(X_test_B[0], axis=0)
pred_B = (model_B.predict(sample_B)[0] > 0.5).astype(int)
actual_B = y_test_B.iloc[0].values

print("\nSubject-wise Prediction Comparison (Sample Student):")
print("--------------------------------------------------------------")
print(f"{'Subject':<45} {'Actual Weak (A)':<15} {'Predicted (A)':<15} {'Actual Weak (B)':<15} {'Predicted (B)':<15}")
print("--------------------------------------------------------------")
for i, sub in enumerate(subjects):
    a_actual = "Yes" if actual_A[i] == 1 else "No"
    a_pred = "Yes" if pred_A[i] == 1 else "No"
    b_actual = "Yes" if actual_B[i] == 1 else "No"
    b_pred = "Yes" if pred_B[i] == 1 else "No"
    print(f"{sub:<45} {a_actual:<15} {a_pred:<15} {b_actual:<15} {b_pred:<15}")
print("--------------------------------------------------------------")


Epoch 1/50


d:\AI Zaki_env\zaki_env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0573 - loss: 0.7082 - val_accuracy: 0.0920 - val_loss: 0.5809
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0876 - loss: 0.5986 - val_accuracy: 0.0220 - val_loss: 0.5499
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0655 - loss: 0.5670 - val_accuracy: 0.0160 - val_loss: 0.5312
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0661 - loss: 0.5496 - val_accuracy: 0.0440 - val_loss: 0.5217
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0540 - loss: 0.5426 - val_accuracy: 0.0240 - val_loss: 0.5159
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0632 - loss: 0.5364 - val_accuracy: 0.0400 - val_loss: 0.5135
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0557 - loss: 0.5324 - val_accuracy: 0.0640 - val_loss: 0.5097
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0683 - loss: 0.5300 - val_accuracy: 0.0860 - val_

Epoch 1/50


d:\AI Zaki_env\zaki_env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0519 - loss: 0.7219 - val_accuracy: 0.0800 - val_loss: 0.5886
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0902 - loss: 0.6096 - val_accuracy: 0.0480 - val_loss: 0.5480
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0663 - loss: 0.5699 - val_accuracy: 0.0660 - val_loss: 0.5273
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0708 - loss: 0.5511 - val_accuracy: 0.0440 - val_loss: 0.5204
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0476 - loss: 0.5427 - val_accuracy: 0.0560 - val_loss: 0.5113
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0619 - loss: 0.5312 - val_accuracy: 0.0340 - val_loss: 0.5065
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0825 - loss: 0.5182 - val_accuracy: 0.0360 - val_loss: 0.5019
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0696 - loss: 0.5214 - val_accuracy: 0.0280 - val_

Model A (Minor-only): Accuracy = 21.00% | Loss = 0.4051
Model B (Full-data): Accuracy = 13.60% | Loss = 0.4063

📊 Model A (Minor-only) Results:
Accuracy: 21.00% | Loss: 0.4051
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

🔍 Overall Classification Report:
              precision    recall  f1-score   support

    Not Weak       0.87      0.94      0.90      5391
        Weak       0.81      0.63      0.71      2109

    accuracy                           0.85      7500
   macro avg       0.84      0.79      0.81      7500
weighted avg       0.85      0.85      0.85      7500

[[5080  311]
 [ 780 1329]]

📊 Model B (Full-data) Results:
Accuracy: 13.60% | Loss: 0.4063
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

🔍 Overall Classification Report:
              precision    recall  f1-score   support

    Not Weak       0.87      0.93      0.90      5401
        Weak       0.80      0.66      0.72      2099

    accuracy                           0.86      7500
   macro avg       0.84      0.79      0.